<a href="https://colab.research.google.com/github/MatheusMol2021/RENEAR/blob/main/ATV2_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matheus Inácio Silva Mol
# Classificador Mnist
Implementar um rede neural que resolva satisfatoriamente (>0.97 acc) o dataset MNIST original.

Download: wget 

http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz 

http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz

http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz

http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

# ATV2_MNIST

Download MNIST DATASET
http://www.dia.fi.upm.es/~lbaumela/PracRF11/MNIST.html

## DATASET MNIST

In [1]:
import numpy as np
import gzip
import urllib.request

In [2]:
# Função para ler o arquivo do MNIST
def read_mnist(images_path, labels_path):
    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(labels), 784)

    return images, labels

In [3]:
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', 'train-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', 'train-labels.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', 'test-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', 'test-labels.gz')

('test-labels.gz', <http.client.HTTPMessage at 0x7f21b7aefe20>)

In [4]:
train_images, train_labels = read_mnist('train-images.gz', 'train-labels.gz')
test_images, test_labels = read_mnist('test-images.gz', 'test-labels.gz')

In [5]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

In [6]:
from tensorflow.keras.utils import to_categorical
train_labels_hot = to_categorical(train_labels)
test_labels_hot = to_categorical(test_labels)

## MODELO

In [7]:
from keras.utils.sidecar_evaluator import optimizer
from prompt_toolkit.shortcuts.progress_bar.base import E
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
import numpy as np

class RedeNeural(BaseEstimator, ClassifierMixin):
  def __init__(self, epochs=5, batch_size=128):
    self.epochs = epochs
    self.batch_size = batch_size

  def fit(self, X, y):
    self.labels, ids = np.unique(y, return_inverse=True)
    yhot = keras.utils.to_categorical(ids)
    self.model = keras.Sequential([
        layers.Dense(512, activation="relu"),
        layers.Dense(yhot.shape[1], activation="softmax")
    ])
    self.model.compile(optimizer="rmsprop",
                       loss="categorical_crossentropy",
                       metrics=["accuracy"])
    self.model.fit(X, yhot, epochs=self.epochs, batch_size=self.batch_size)
    return self

  def predict(self, X, y=None):
    probabilities = self.model.predict(X)
    ypred = self.labels[np.argmax(probabilities, axis=1)]
    return ypred

modelo = RedeNeural()
modelo.fit(train_images, train_labels)
ypred = modelo.predict(test_images)
accuracy_score(test_labels, ypred)

Epoch 1/5
469/469 [==============================] - 10s 19ms/step - loss: 0.2671 - accuracy: 0.9229
Epoch 2/5
469/469 [==============================] - 5s 10ms/step - loss: 0.1095 - accuracy: 0.9676
Epoch 3/5
469/469 [==============================] - 6s 13ms/step - loss: 0.0725 - accuracy: 0.9781
Epoch 4/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0522 - accuracy: 0.9846
Epoch 5/5
313/313 [==============================] - 1s 4ms/step


0.9794

In [8]:
train_images = train_images.reshape((60000, 28 * 28))
test_images = test_images.reshape((10000, 28 * 28))

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

modelo = Pipeline([
    ("scaler", MinMaxScaler()), 
    ("ann", RedeNeural())
])
modelo.fit(train_images, train_labels)
ypred = modelo.predict(test_images)
accuracy_score(test_labels, ypred)

Epoch 1/5
469/469 [==============================] - 5s 10ms/step - loss: 0.2642 - accuracy: 0.9236
Epoch 2/5
469/469 [==============================] - 5s 10ms/step - loss: 0.1080 - accuracy: 0.9682
Epoch 3/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0716 - accuracy: 0.9786
Epoch 4/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0519 - accuracy: 0.9845
Epoch 5/5
313/313 [==============================] - 1s 3ms/step


0.9799